In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('Feature_Selection.csv')

In [3]:
df.head()

,x.aidtst3,employ1,income2,weight2,height3,children,veteran3,blind,renthom1,sex1,...,x.denvst3,x.prace1,x.mrace1,x.exteth3,x.asthms1,x.michd,x.ltasth1,x.casthm1,x.state,havarth3
0,2.0,1.0,8.0,138.0,504.0,2.0,2.0,2.0,1.0,2,...,1,1.0,1.0,1,3,2.0,1,1,27.0,2
1,1.0,1.0,4.0,240.0,600.0,88.0,2.0,1.0,2.0,1,...,2,1.0,1.0,2,3,2.0,1,1,72.0,2
2,1.0,1.0,6.0,120.0,504.0,88.0,2.0,2.0,1.0,2,...,1,6.0,6.0,2,3,2.0,1,1,31.0,2
3,2.0,7.0,77.0,133.0,502.0,88.0,2.0,2.0,1.0,2,...,1,1.0,1.0,2,3,2.0,1,1,45.0,2
4,9.0,7.0,8.0,170.0,505.0,88.0,2.0,2.0,1.0,2,...,1,1.0,1.0,1,3,2.0,1,1,24.0,2


In [10]:
df.shape

(11933, 108)

In [9]:
df.columns

Index(['x.aidtst3', 'employ1', 'income2', 'weight2', 'height3', 'children',
       'veteran3', 'blind', 'renthom1', 'sex1',
       ...
       'x.denvst3', 'x.prace1', 'x.mrace1', 'x.exteth3', 'x.asthms1',
       'x.michd', 'x.ltasth1', 'x.casthm1', 'x.state', 'havarth3'],
      dtype='object', length=108)

In [5]:
X= df.drop(columns=["havarth3"])
y= df['havarth3']

In [4]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

In [13]:
uni = SelectKBest(score_func = f_classif, k = 10)
fit = uni.fit(X, y)

In [14]:
best = fit.transform(X)

In [15]:
X.columns[fit.get_support(indices=True)].tolist()

['employ1',
 'rmvteth4',
 'genhlth',
 'x.age.g',
 'x.age80',
 'x.ageg5yr',
 'x.age65yr',
 'x.rfhlth',
 'x.phys14d',
 'x.hcvu651']

#### PCA

In [16]:
from sklearn.decomposition import PCA

pca = PCA(n_components = 10)
x = pca.fit_transform(X)

In [17]:
model=PCA(n_components=10).fit(x)
n_pcs = model.components_.shape[0]
most_important = [np.abs(model.components_[i]).argmax() for i in range(n_pcs)]
df.columns[most_important]

Index(['x.aidtst3', 'employ1', 'income2', 'weight2', 'height3', 'children',
       'veteran3', 'blind', 'renthom1', 'sex1'],
      dtype='object')

In [18]:
model.components_

array([[ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [-0.00000000e+00,  1.00000000e+00,  8.10683999e-15,
         7.24904126e-19, -1.10595757e-18, -1.35790982e-18,
        -5.52223685e-18,  1.74804837e-17, -3.68769169e-18,
        -2.30805335e-18],
       [ 0.00000000e+00, -8.08090507e-15,  1.00000000e+00,
         5.49541958e-17, -3.86504098e-17,  3.11128141e-17,
        -3.05142811e-17, -3.82038884e-16, -6.97021476e-19,
         2.67224144e-17],
       [ 0.00000000e+00, -1.62404599e-17,  1.73483951e-16,
         1.00000000e+00, -2.12328023e-15,  3.08818919e-16,
        -2.97492126e-16, -2.36844123e-16,  4.84968828e-17,
         6.82797980e-18],
       [ 0.00000000e+00, -1.49273612e-18,  8.51818386e-17,
         1.84402515e-15,  1.00000000e+00, -6.80037003e-16,
         5.22565894e-16,  3.34567143e-16, -5.44658333e-18,
        -8.

#### Correlation Matrix

In [19]:
cor = df.corr()
cor_target = abs(cor["havarth3"])
relevant_features = cor_target[cor_target > 0.2]
relevant_features.index

Index(['employ1', 'genhlth', 'x.age.g', 'x.age80', 'x.ageg5yr', 'x.age65yr',
       'x.hcvu651', 'havarth3'],
      dtype='object')

#### Wrapper method

In [22]:
import statsmodels.api as sm
X_new = sm.add_constant(X)
model = sm.OLS(y, X_new).fit()
model.pvalues

const        2.132756e-01
x.aidtst3    6.269686e-01
employ1      4.025786e-20
income2      3.931291e-04
weight2      2.122768e-01
                 ...     
x.asthms1    3.445036e-01
x.michd      3.478433e-01
x.ltasth1    3.081917e-03
x.casthm1    9.802652e-01
x.state      6.724318e-01
Length: 108, dtype: float64

In [24]:
selected_features = list(X.columns)
pmax = 1
while (len(selected_features)>0):
    p= []
    X_new = X[selected_features]
    X_new = sm.add_constant(X_new)
    model = sm.OLS(y,X_new).fit()
    p = pd.Series(model.pvalues.values[1:],index = selected_features)      
    pmax = max(p)
    feature_pmax = p.idxmax()
    if(pmax>0.05):
        selected_features.remove(feature_pmax)
    else:
        break
selected_features

['employ1',
 'income2',
 'blind',
 'sex1',
 'pneuvac4',
 'diffwalk',
 'diffdres',
 'smoke100',
 'rmvteth4',
 'physhlth',
 'menthlth',
 'hlthpln1',
 'genhlth',
 'persdoc2',
 'checkup1',
 'addepev2',
 'chcscncr',
 'asthma3',
 'qstlang',
 'x.metstat',
 'htin4',
 'wtkg3',
 'x.age.g',
 'x.ageg5yr',
 'x.age65yr',
 'x.chldcnt',
 'x.incomg',
 'x.rfseat3',
 'x.rfsmok3',
 'x.urbstat',
 'x.llcpwt2',
 'x.rfhlth',
 'x.imprace',
 'x.wt2rake',
 'x.strwt',
 'x.phys14d',
 'x.hcvu651',
 'x.denvst3',
 'x.prace1',
 'x.mrace1',
 'x.ltasth1']